In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import torch

from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, models, optimizers
from tqdm import tqdm

In [286]:
data_merge = pd.read_csv('data_merge.csv',index_col=[0], parse_dates=[4])
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608 entries, 0 to 607
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   text            608 non-null    object        
 1   CGN             608 non-null    object        
 2   meta            608 non-null    object        
 3   date            608 non-null    datetime64[ns]
 4   candidate_type  608 non-null    object        
 5   candidate_name  607 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 33.2+ KB


In [287]:
data_merge.head(3)

,text,CGN,meta,date,candidate_type,candidate_name
0,\n20230731_12345\nКитайский документ 22-9trans...,202305310008,\nДата выхода: 2023-08-02\nВ 2023 году в Китае...,2023-08-02,Владелец,SMS
1,\n20230731_12345\nКитайский документ 85-8trans...,202211140006,\nДата выхода: 2023-01-16\nЗакупка оборудовани...,2023-01-16,Разработчик,East Electric Group Eastern Electric Company
2,\n20230731_12345\nКитайский документ 8-8trans....,202307050014,\nДата выхода: 2023-08-24\nВ пункте 1 проекта ...,2023-08-24,Разработчик,China Electric Industrial Co. Ltd.


In [288]:
part1 = pd.read_csv('datasets/part_1_BIG_MODELfor_merge.csv')
part1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location        581 non-null    object 
 1   product         571 non-null    object 
 2   tender_number   574 non-null    object 
 3   tender_name     148 non-null    object 
 4   tender_company  339 non-null    object 
 5   quantity        1 non-null      float64
dtypes: float64(1), object(5)
memory usage: 28.6+ KB


In [289]:
part1 = part1.drop(['tender_name', 'quantity'], axis=1)
part1.head(3)

,location,product,tender_number,tender_company
0,"Гуандун, Чжунцин, Каннинг",оборудование для замены коробок,CGN-202305310008,Новая энергетическая компания
1,"Немонгольский автономный район, УНИТА, правый ...",оборудование для оптических камер,CGN-202211140006,Новая энергетическая компания
2,"Синьцзян-Уйгурский автономный район и Тэгу, ок...",1 млн. кВт оптико-волнового поля PC,CGN-202307050014,China Solar Energy Successing Ltd.


In [290]:
part2 = pd.read_csv('datasets/part_2_for_merge.csv', parse_dates=[4, 5])
part2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   description    414 non-null    object
 1   requirements   360 non-null    object
 2   stages         258 non-null    object
 3   delivery_time  316 non-null    object
 4   start_date     230 non-null    object
 5   end_date       205 non-null    object
dtypes: object(6)
memory usage: 28.6+ KB


/tmp/ipykernel_78417/3156127547.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  part2 = pd.read_csv('datasets/part_2_for_merge.csv', parse_dates=[4, 5])
/tmp/ipykernel_78417/3156127547.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  part2 = pd.read_csv('datasets/part_2_for_merge.csv', parse_dates=[4, 5])


In [291]:
part2.head(3)

,description,requirements,stages,delivery_time,start_date,end_date
0,"Проект <<Хиннин>>, расположенный в городе Цзян...",Этот тендер будет состоять из первого сегмента...,четыре сегмента: 1-й пункт: кузов; 2-й участок...,Предварительный 15 августа 2023 года; срок око...,15.08.2023,NaN
1,Закупка оборудования для фотоаппаратов УНИТА в...,Участники торгов предоставили три комплекта ка...,2,до 10 июня 2023 года,NaN,NaN
2,Общий контрактный проект PC в 1 млн кВт-диапаз...,Упаковка 1: первый этап проекта генерального п...,четыре сегмента,Планируемый период работы составляет с 15 авгу...,15.08.2023,30.12.2023


In [292]:
price = pd.read_csv('datasets/clear_price.csv')
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   598 non-null    float64
dtypes: float64(1)
memory usage: 4.9 KB


In [293]:
data_full = data_merge[['CGN','text']].join(part1.set_axis(data_merge.index)).join(part2.set_axis(data_merge.index))
data_full['price'] = price

In [294]:
data_full.head(3)

,CGN,text,location,product,tender_number,tender_company,description,requirements,stages,delivery_time,start_date,end_date,price
0,202305310008,\n20230731_12345\nКитайский документ 22-9trans...,"Гуандун, Чжунцин, Каннинг",оборудование для замены коробок,CGN-202305310008,Новая энергетическая компания,"Проект <<Хиннин>>, расположенный в городе Цзян...",Этот тендер будет состоять из первого сегмента...,четыре сегмента: 1-й пункт: кузов; 2-й участок...,Предварительный 15 августа 2023 года; срок око...,15.08.2023,NaN,3591735.0
1,202211140006,\n20230731_12345\nКитайский документ 85-8trans...,"Немонгольский автономный район, УНИТА, правый ...",оборудование для оптических камер,CGN-202211140006,Новая энергетическая компания,Закупка оборудования для фотоаппаратов УНИТА в...,Участники торгов предоставили три комплекта ка...,2,до 10 июня 2023 года,NaN,NaN,64806000.0
2,202307050014,\n20230731_12345\nКитайский документ 8-8trans....,"Синьцзян-Уйгурский автономный район и Тэгу, ок...",1 млн. кВт оптико-волнового поля PC,CGN-202307050014,China Solar Energy Successing Ltd.,Общий контрактный проект PC в 1 млн кВт-диапаз...,Упаковка 1: первый этап проекта генерального п...,четыре сегмента,Планируемый период работы составляет с 15 авгу...,15.08.2023,30.12.2023,666628967.0


In [362]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608 entries, 0 to 607
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CGN             608 non-null    object 
 1   text            608 non-null    object 
 2   location        581 non-null    object 
 3   product         571 non-null    object 
 4   tender_number   574 non-null    object 
 5   tender_company  339 non-null    object 
 6   description     414 non-null    object 
 7   requirements    360 non-null    object 
 8   stages          258 non-null    object 
 9   delivery_time   316 non-null    object 
 10  start_date      230 non-null    object 
 11  end_date        205 non-null    object 
 12  price           598 non-null    float64
dtypes: float64(1), object(12)
memory usage: 82.7+ KB


In [369]:
def count_nulls(row):
    return ((row['location'] == 'не указано') +
           (row['product'] == 'не указано') +
           (row['tender_company'] == 'не указано') +
           (row['description'] == 'не указано') +
           (row['requirements'] == 'не указано') +
           (row['stages'] == 'не указано'))

In [370]:
data_full_nulls = data_full.copy()[['text', 'location', 'product', 'tender_company', 'description', 'requirements', 'stages','price']]
data_full_nulls = data_full_nulls[data_full_nulls['price'] < 10**10]
data_full_nulls[['text', 'location', 'product', 'tender_company', 'description', 'requirements', 'stages']] = data_full_nulls[['text', 'location', 'product', 'tender_company', 'description', 'requirements', 'stages']].fillna('не указано')

In [371]:
data_full_nulls['nulls'] = data_full_nulls.apply(count_nulls, axis=1)

In [372]:
data_full_nulls = data_full_nulls[data_full_nulls['nulls'] < 4].reset_index()

In [373]:
data_full_nulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           499 non-null    int64  
 1   text            499 non-null    object 
 2   location        499 non-null    object 
 3   product         499 non-null    object 
 4   tender_company  499 non-null    object 
 5   description     499 non-null    object 
 6   requirements    499 non-null    object 
 7   stages          499 non-null    object 
 8   price           499 non-null    float64
 9   nulls           499 non-null    int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 39.1+ KB


In [374]:
class VectorStore:
    def __init__(self, model, size):
        self.documents = []
        self.embeddings = np.empty(size)
        self.model = model  

    def add_to_store(self, document):
        self.documents.append(document)
        embedding = self.model.encode(document, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False, batch_size=32)
        self.embeddings = np.vstack((self.embeddings, embedding))

In [375]:
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# model_name = 'embaas/sentence-transformers-e5-large-v2'
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
# model_name = 'cointegrated/rut5-base'
# model_name = 'sentence-transformers/LaBSE'

In [376]:
device = torch.device("cuda")
device

device(type='cuda')

In [377]:
lm_model = SentenceTransformer(model_name, device=device)
lm_model[1].pooling_mode_mean_tokens = False
lm_model[1].pooling_mode_cls_token = True
lm_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [378]:
data_full_nulls.columns

Index(['index', 'text', 'location', 'product', 'tender_company', 'description',
       'requirements', 'stages', 'price', 'nulls'],
      dtype='object')

In [379]:
cols = ['text', 'location', 'product', 'tender_company', 'description', 'requirements', 'stages']

In [380]:
stores = []
for col in cols:
    store = VectorStore(lm_model, size=(0, lm_model[1].word_embedding_dimension)) 
    for j in tqdm(range(data_full_nulls.shape[0])):
        store.add_to_store(data_full_nulls[col][j])
    stores.append(store)

100%|████████████████████████████████████████| 499/499 [00:03<00:00, 149.37it/s]


In [381]:
stores[0].embeddings.shape

(499, 384)

In [382]:
data_full_emb = pd.DataFrame(stores[0].embeddings)
data_full_emb.columns = data_full_emb.columns.map(lambda x: str(x) + '_0')
for i in range(1, len(stores)):
    temp_df = pd.DataFrame(stores[i].embeddings)
    temp_df.columns = temp_df.columns.map(lambda x: str(x) + f'_{i}')
    data_full_emb = data_full_emb.join(temp_df.set_axis(data_full_emb.index), lsuffix=f'_{i}')
data_full_emb.shape[1]

2688

In [383]:
data_full_emb['price'] = data_full_nulls['price']

In [384]:
train, valid = train_test_split(data_full_emb, test_size=0.2, random_state=42)
valid, test = train_test_split(valid, test_size=0.5, random_state=42)
train.shape, valid.shape, test.shape

((399, 2689), (50, 2689), (50, 2689))

In [385]:
X_train = train.drop('price', axis=1)
y_train = train['price']
X_valid = valid.drop('price', axis=1)
y_valid = valid['price']
X_test = test.drop('price', axis=1)
y_test = test['price']

In [386]:
y_valid.mean()

169442293.84300002

In [410]:
seq_model = models.Sequential([
    layers.Dense(units=(64), input_shape=(X_train.shape[1],), activation='relu'),
    layers.Dense(units=(32), activation='relu'),
    layers.Dense(units=1)
])

In [411]:
%%time
seq_model.compile(optimizer='Adam', loss='mse', metrics=['mae'])
seq_model.fit(X_train, y_train, epochs=500, verbose=0)

CPU times: user 15.5 s, sys: 3.01 s, total: 18.5 s
Wall time: 5.85 s


In [412]:
pred_valid = seq_model.predict(X_valid, verbose=0)
print('MAE:', mean_absolute_error(y_valid, pred_valid).round())
(y_valid-pred_valid[0]).abs().describe() / 10 ** 6

MAE: 163512375.0


count       0.000050
mean      163.237506
std       734.376494
min         0.254081
25%         5.468040
50%         9.553063
75%        33.858919
max      5077.715277
Name: price, dtype: float64

In [407]:
(y_valid-y_valid.median()).abs().describe() / 10 ** 6

count       0.000050
mean      163.175234
std       734.122267
min         0.124558
25%         5.582165
50%        10.787424
75%        32.624558
max      5076.480916
Name: price, dtype: float64

In [395]:
reg = LinearRegression().fit(X_train, y_train)

In [396]:
pred_valid = reg.predict(X_valid)
mean_absolute_error(y_valid, pred_valid)

1.4397659858565997e+17

In [397]:
%%time
cat_model = CatBoostRegressor(iterations=1000)
# Fit model
cat_model.fit(X_train, y_train, verbose=250)
# Get predictions
preds = cat_model.predict(X_valid)

Learning rate set to 0.035411
0:	learn: 337653325.7884295	total: 92.8ms	remaining: 1m 32s
250:	learn: 70345433.0928211	total: 20.8s	remaining: 1m 2s
500:	learn: 13763609.1892927	total: 41.4s	remaining: 41.2s
750:	learn: 3441022.0623790	total: 1m 2s	remaining: 20.6s
999:	learn: 1396137.0637531	total: 1m 22s	remaining: 0us
CPU times: user 15min 42s, sys: 5.6 s, total: 15min 48s
Wall time: 1min 23s


In [398]:
mean_absolute_error(y_valid, preds)

174091584.18911964

### Тест

In [413]:
pred_test = seq_model.predict(X_test, verbose=0)
(y_test-pred_test[0]).abs().describe() / 10 ** 6

count      0.000050
mean      69.723868
std      168.415750
min        0.279403
25%        4.827568
50%        8.356079
75%       31.292074
max      659.058960
Name: price, dtype: float64

In [414]:
(y_test-y_test.median()).abs().describe() / 10 ** 6

count      0.000050
mean      69.601445
std      167.232454
min        0.185380
25%        6.627749
50%       10.900000
75%       28.046129
max      655.813014
Name: price, dtype: float64